# DEWAN LAB MICROENDOSCOPE ANALYSIS TEMPLATE
##### This script uses functions from the Inscopix API to: 1) preprocess video files (i.e., deinterleaving, bandpass filtering and downsampling), 2) motion correction, 3) dF/F normalization, 4) identify cells using CNMFE algorithm, 5) produce a max projection image, and 6) export files. 
#### Copy this Jupyter Notebook into a folder specific for the animal/session, rename it according to the animal/session (e.g., VGAT4_Session1_InscopixAnalysis), and adjust user configurables

### Step 1: Import Modules

In [ ]:
import time
import isx 
#  Inscopix API; Navigate to C:\Program Files\Inscopix\Data Processing\ ; run pip install .
from pathlib import Path
from dewan_calcium.helpers import IO, parse_json
from dewan_calcium.helpers.project_folder import ProjectFolder
# Must install Dewan Lab Calcium Imaging Toolbox found here: https://github.com/OlfactoryBehaviorLab/dewan_calcium

### Step 2: Adjust User Configurables

In [ ]:
experiment_name = 'NAME_GOES_HERE'
override_data_dir = '/mnt/d/Test_Data/Odor/VGLUT-20'

# Set to True to delete all the intermediary Inscopix Files
cleanup_interim_files = False

# Focal plane(s) where cells were recorded 
# If set to -1, the session.json file will be read to automatically set the focal plane; replace -1 to manually set focal planes
# If these values do not match the efocus used in the exp, program will stop
focal_planes = [-1]

# Downsample Settings
spatial_downsample_factor = 4 # Factor of 4 is recommended for CNMFE
temporal_downsample_factor = 1 # CNMFE is best suited for temporal resolution <= 10Hz; adjust according to floor(Frame Rate / 10); we choose to downsample all videos to 10Hz

# Spatial Bandpass Settings
spatial_bp_low_cutoff = 0.005
spatial_bp_high_cutoff = 0.500

# Motion Correction Settings
max_translation = 20
motion_c_low_bandpass_cutoff = None
motion_c_high_bandpass_cutoff = None

# CNMFE Settings
cell_diameter = 7
merge_threshold = 0.7
gaussian_kernel_size = 3
output_unit_type = 0
minimum_correlation = 0.9
minimum_pnr = 15

In [ ]:
if override_data_dir is not None:
    project_folder = ProjectFolder(project_dir=override_data_dir)
else:
    project_folder = ProjectFolder()

### Step 3: Get directories and video files

In [ ]:
interim_output_directory = project_folder.inscopix_dir.interim_file_dir.path

output_directory = project_folder.inscopix_dir.path

video_files = project_folder.raw_data_dir.raw_recordings
gpio_input_path = project_folder.raw_data_dir.raw_GPIO_path
session_json_path = project_folder.raw_data_dir.session_json_path
video_base = gpio_input_path.stem

In [ ]:
# 3C: Parse Focal Planes out of JSON File

if focal_planes == [-1]:
    gain, led_power, focal_planes = parse_json.get_session_settings(session_json_path)
    
    focal_planes = [focal_planes]  # Wrap focal planes in a list
    print(f'Focal Plane(s) set to: {focal_planes}')
    print(f'Recording Gain: {gain}')
    print(f'Recording LED Power: {led_power}')

### STEP 4: Deinterleaving - Split multiplane recordings into individual files for each focal plane

In [ ]:
if len(focal_planes) == 1:  # If there is only one focal plane, we don't need to deinterleave!
    videos_to_process = video_files
else:  # If there is more than one focal plane, deinterleave!
    deinterleave_paths = IO.generate_deinterleaved_video_paths(video_files, interim_output_directory, focal_planes)
    if IO.check_files(video_files, deinterleave_paths):
        print(f"Starting deinterleaving process. Creating videos for focal planes: {focal_planes}")
        try:
            isx.de_interleave(video_files, deinterleave_paths, focal_planes)
            print("Finished Deinterleaving!")
        except Exception as exception:  # This isn't great form, but Inscopix throws a very generic Exception
            print(exception.args[0])
            print("It is likely the number of focal planes provided do not match the number in the video!")
    videos_to_process = deinterleave_paths

### Step 5A-E: Process video files using standard analysis pipeline
#### Dewan Lab Specific Instructions: The steps below can be run on R2D2

In [ ]:
for video in videos_to_process:
    print(f'Processing file: {video}')
    video = str(video)

    # STEP 5A: PREPROCESS VIDEO FILES WITH TEMPORAL AND SPATIAL DOWNSAMPLING
    preprocessed_files = IO.make_isx_path([video], interim_output_directory, 'PP')
    if IO.check_files([video], preprocessed_files):
        # Checks if input file(s) are missing, or output file(s) already exist, function returns True if this step can be run
        print('Starting Preprocessing...')
        isx.preprocess(video, preprocessed_files, temporal_downsample_factor=temporal_downsample_factor, spatial_downsample_factor=spatial_downsample_factor)
        print('Preprocessing Finished!')


    # STEP 5B: APPLY BANDPASS FILTER TO VIDEO FILES
    bandpass_files = IO.make_isx_path(preprocessed_files, interim_output_directory, 'BP')
    if IO.check_files(preprocessed_files, bandpass_files):
        print('Starting Bandpass Filtering...')
        isx.spatial_filter(preprocessed_files, bandpass_files, low_cutoff=spatial_bp_low_cutoff, high_cutoff=spatial_bp_high_cutoff)
        print('Bandpass Filtering Finished!')


    # STEP 5C.1: GENERATE MEAN IMAGE TO USE AS THE REFERENCE FRAME FOR MOTION CORRECTION
    mean_projection_file = IO.make_isx_path([video], interim_output_directory, 'mean_image')
    if IO.check_files(bandpass_files, mean_projection_file):
        print('Creating Mean Image...')
        isx.project_movie(bandpass_files, mean_projection_file[0], stat_type='mean')
        print('Mean Image Created!')

    # STEP 5C.2: APPLY MOTION CORRECTION
    motion_correction_files = IO.make_isx_path(bandpass_files, interim_output_directory, 'MC')
    translation_file = IO.make_isx_path(motion_correction_files, interim_output_directory, 'translations', 'csv')[0]
    crop_rect_file = IO.make_isx_path([video], interim_output_directory, 'crop_rect', 'csv')[0]
    if IO.check_files(bandpass_files, [motion_correction_files, translation_file, crop_rect_file]):
        print('Starting Motion Correction...')
        isx.motion_correct(bandpass_files, motion_correction_files, max_translation=max_translation, reference_file_name=mean_projection_file[0],
                           low_bandpass_cutoff=motion_c_low_bandpass_cutoff, high_bandpass_cutoff=motion_c_high_bandpass_cutoff,
                           output_translation_files=translation_file, output_crop_rect_file=crop_rect_file)
        print('Motion Correction Finished!')


    # STEP 5D: DELTA F/F NORMALIZATION
    dff_files = IO.make_isx_path(motion_correction_files, interim_output_directory, 'DFF')
    if IO.check_files(motion_correction_files, dff_files):
        print('Calculating DF/F...')
        isx.dff(motion_correction_files, dff_files, f0_type='mean')
        print('Calculating DF/F Finished!')


    # STEP 5E.1: MAKE A MAX PROJECTION IMAGE OF THE RECORDING (NOTE IT IS DOWNSAMPLED)
    max_projection_file = IO.make_isx_path(dff_files, interim_output_directory, 'MAX_PROJ')
    if IO.check_files([dff_files[0]], max_projection_file):
        print("Starting Max Projection Creation. This might take a while...")
        isx.project_movie(dff_files[0], max_projection_file[0], stat_type='max')
    # STEP 5E.2: CONVERT ISXD MAX PROJECTION IMAGE TO A TIFF FILE
    tiff_path = IO.make_isx_path(max_projection_file, output_directory, addition='low-resolution', extension='tiff')
    if IO.check_files([max_projection_file[0]], tiff_path):
        isx.export_isxd_image_to_tiff(max_projection_file[0], tiff_path[0])
        print("Max Projection Creation finished!")

### Step 5F-I: Process video files using standard analysis pipeline
#### Dewan Lab Specific Instructions: The steps below should be run locally if an odor experiment or very large recording
Run steps 1-4, then skip here after the first part of step 5 has finished

In [ ]:
for video in videos_to_process:
    # STEP 5F.1: Generate file paths for CNMFE
    preprocessed_files = IO.make_isx_path([video], interim_output_directory, 'PP')
    bandpass_files = IO.make_isx_path(preprocessed_files, interim_output_directory, 'BP')
    motion_correction_files = IO.make_isx_path(bandpass_files, interim_output_directory, 'MC')
    dff_files = IO.make_isx_path(motion_correction_files, interim_output_directory, 'DFF')
    translation_file = IO.make_isx_path(motion_correction_files, interim_output_directory, 'translations', 'csv')[0]
    crop_rect_file = IO.make_isx_path([video], interim_output_directory, 'crop_rect', 'csv')[0]
    max_projection_file = IO.make_isx_path(dff_files, interim_output_directory, 'MAX_PROJ')

    # STEP 5F.2: RUN CNFME CELL DETECTION ALGORITHM
    cnmfe_files = IO.make_isx_path(motion_correction_files, output_directory, 'CNMFE')
    if IO.check_files(motion_correction_files, cnmfe_files):
        print('Running CNMFe (This Will Take a Long Time)...')
        start_time = time.time()
        isx.run_cnmfe(motion_correction_files, cnmfe_files, str(interim_output_directory), cell_diameter=cell_diameter, merge_threshold=merge_threshold, gaussian_kernel_size=gaussian_kernel_size,
                      output_unit_type='df', min_corr=minimum_correlation, min_pnr=minimum_pnr, processing_mode='parallel_patches', num_threads=8)  #  No DF/F on CNMFE Files
        end_time = time.time()
        print(f'CNMFE Finished! Total Elapsed Time: {round((end_time - start_time) / 60, 2)} (min)')
    
    if IO.check_files(cnmfe_files, None):
        # If CNMFE finds no cells, it does not create an output file. The following functions will crash without that file.
        # STEP 5G: APPLY CELL CONTOURS (OUTLINES)
        contour_files = IO.make_isx_path(dff_files, output_directory, 'AC')
        if IO.check_files(dff_files, contour_files):
            print('Applying Cell Contours...')
            isx.apply_cell_set(dff_files, cnmfe_files[0], contour_files, 0)
            print('Finished Applying Cell Contours!')
    
    
        # STEP 5H: EXPORT DATA FILES
        tiff_image_path = IO.make_isx_path([video], output_directory, extension='tiff')[0]
        trace_file_path = IO.make_isx_path([video], output_directory, 'TRACES', 'csv')[0]
        props_file_path = IO.make_isx_path([video], output_directory, 'props', 'csv')[0]
        json_file_path = IO.make_isx_path([video], output_directory, 'CONTOURS', 'json')[0]
        gpio_output_path = IO.make_isx_path([video], output_directory, 'GPIO', 'csv')[0]
        if IO.check_files(contour_files, [props_file_path, tiff_image_path, trace_file_path, gpio_output_path]):
            print('Exporting Traces and Props...')
            isx.export_cell_set_to_csv_tiff(contour_files, trace_file_path, tiff_image_path, time_ref='unix', output_props_file=props_file_path)
            print('Exporting Cell Contours...')
            isx.export_cell_contours(cnmfe_files[0], json_file_path)
            print("Exporting GPIO to csv...")
            isx.export_gpio_set_to_csv(str(gpio_input_path), str(gpio_output_path), time_ref='unix')
            print("File export finished!")
    
        print(f'Processing Finished for {video}!')
    
    
        # STEP 5I (OPTIONAL): DELETE ALL INTERIM FILES
        if cleanup_interim_files:
            for each in preprocessed_files, bandpass_files, motion_correction_files, dff_files, max_projection_file:
                Path(each[0]).unlink()
    
    else:
        print("CNMFE Found no cells. There is no data to export!")


### STEP 6:  Produce a high resolution max projection image; This must be done for the manual curation of cells
#### Must run steps 1-4 first to load configuration, file paths, and deinterleave if needed


In [ ]:
for video in videos_to_process:
    print(f'Processing file: {video}')
    video = str(video)

    # STEP 6A: PREPROCESS VIDEO FILES WITH NO DOWNSAMPLING
    preprocessed_files = IO.make_isx_path([video], interim_output_directory, 'HD-PP')
    if IO.check_files(None, preprocessed_files):
        # Checks if input files are missing, or output file(s) already exist, function returns True if this step can be run
        print('Starting Preprocessing...')
        isx.preprocess(video, preprocessed_files)
        print('Preprocessing Finished!')


    # STEP 6B: APPLY BANDPASS FILTER TO HD VIDEO FILES
    bandpass_files= IO.make_isx_path(preprocessed_files, interim_output_directory, 'BP')
    if IO.check_files(preprocessed_files, bandpass_files):
        print('Starting Bandpass Filtering...')
        isx.spatial_filter(preprocessed_files, bandpass_files, low_cutoff=spatial_bp_low_cutoff, high_cutoff=spatial_bp_high_cutoff)
        print('Bandpass Filtering Finished!')


    # STEP 6C.1: GENERATE MEAN IMAGE TO USE AS THE REFERENCE FRAME FOR MOTION CORRECTION
    mean_projection_file = IO.make_isx_path([video], interim_output_directory, 'HD-mean_image')
    if IO.check_files(bandpass_files, mean_projection_file):
        print('Creating Mean Image...')
        isx.project_movie(bandpass_files, mean_projection_file[0], stat_type='mean')

    # STEP 6C.2: APPLY MOTION CORRECTION TO HD VIDEO FILES
    motion_correction_files = IO.make_isx_path(bandpass_files, interim_output_directory, 'MC')
    translation_file = IO.make_isx_path(motion_correction_files, interim_output_directory, 'translations', 'csv')[0]
    crop_rect_file = IO.make_isx_path([video], interim_output_directory, 'HD-crop_rect', 'csv')[0]
    
    if IO.check_files(bandpass_files, [motion_correction_files, translation_file, crop_rect_file]):
        print('Starting Motion Correction...')
        isx.motion_correct(bandpass_files, motion_correction_files, max_translation=max_translation, reference_file_name=mean_projection_file[0],
                           low_bandpass_cutoff=motion_c_low_bandpass_cutoff, high_bandpass_cutoff=motion_c_high_bandpass_cutoff,
                           output_translation_files=translation_file, output_crop_rect_file=crop_rect_file)
        print('Motion Correction Finished!')



    # STEP 6D: DELTA F/F NORMALIZATION
    dff_files = IO.make_isx_path(motion_correction_files, interim_output_directory, 'DFF')
    if IO.check_files(motion_correction_files, dff_files):
        print('Calculating DF/F...')
        isx.dff(motion_correction_files, dff_files, f0_type='mean')
        print('Calculating DF/F Finished!')


    # STEP 5E.1: MAKE A HD MAX PROJECTION IMAGE OF THE RECORDING
    max_projection_file = IO.make_isx_path(dff_files, interim_output_directory, 'MAX_PROJ')
    if IO.check_files([dff_files[0]], max_projection_file):
        print("Starting Max Projection Creation. This might take a while...")
        isx.project_movie(dff_files[0], max_projection_file[0], stat_type='max')
        print("Max Projection Creation finished!")

    # STEP 6E.2: CONVERT ISXD MAX PROJECTION IMAGE TO A TIFF FILE
    tiff_path = IO.make_isx_path(max_projection_file, output_directory, extension='tiff')
    if IO.check_files([max_projection_file[0]], tiff_path):
        print("Converting Max Projection to tiff file!")
        isx.export_isxd_image_to_tiff(max_projection_file[0], tiff_path[0])

    print(f"HD Max Projection creation for {video} complete!")

    # STEP 6F (OPTIONAL): DELETE ALL INTERIM FILES
    if cleanup_interim_files:
        for each in preprocessed_files, bandpass_files, motion_correction_files, dff_files, max_projection_file:
            Path(each[0]).unlink()

### STEP 7 (OPTIONAL): Produce a high resolution dF/F movie
#### Must run steps 1-4 first to load configuration, file paths, and deinterleave if needed


In [ ]:
# STEP 7A.1: SELECT VIDEO FILE TO TRIM AND EXPORT IN HD. IF THERE IS ONLY ONE FILE, IT WILL AUTOMATICALLY BE SELECTED
if len(videos_to_process) == 1:
    file_number_to_process = 0  # If there is only one video, we want the zero'th index.
else:
    video_options = ['Enter the number of the video you wish to export:\n']
    for i, each in enumerate(videos_to_process):
        video_options.append(f'({i}) {each}\n')
    video_options = ''.join(video_options)

    file_number_to_process = int(input(video_options))

video_file_to_trim = str(videos_to_process[file_number_to_process]) # Convert Path object to string

movie = isx.Movie.read(video_file_to_trim)  # Load video and get the number of total frames.
num_frames = movie.timing.num_samples   # This is equal to TOTAL FRAMES / N FOCAL PLANES
del movie # Free up the memory taken by loading this giant file

# STEP 7A.2: ENTER THE START AND END FRAMES FOR THE TRIMMED VIDEO
start_frame = int(input(f"Enter the START frame out of {num_frames}: "))
end_frame = int(input(f"Enter the END frame out of {num_frames}: "))

if start_frame < 0:
    start_frame = 1
if end_frame == -1:
    end_frame = num_frames


frames_to_trim = [(0, start_frame), (end_frame, num_frames)]
# Trim range from beginning of video to the start frame, and the end frame to end of video


# STEP 7B: TRIM THE SELECTED VIDEO TO THE ENTERED FRAMES
trimmed_video_path = IO.make_isx_path([video_file_to_trim], interim_output_directory, addition=f'HD-TRIM-[{start_frame}-{end_frame}]')
if IO.check_files([video_file_to_trim], trimmed_video_path):
    print(f"Trimming video to [{start_frame, end_frame}]")
    isx.trim_movie(video_file_to_trim, trimmed_video_path, frames_to_trim)
    print("Done trimming!")

movie_stem = Path(video_file_to_trim).stem
movie_filename = f'{movie_stem}-HD-[{start_frame}-{end_frame}].mp4'
movie_file_path = IO.make_isx_path([movie_filename], output_directory, extension='mp4')

print(f'Processing file: {trimmed_video_path[0]}')


# STEP 7C: PREPROCESS TRIMMED VIDEO FILES WITH NO DOWNSAMPLING
preprocessed_files = IO.make_isx_path(trimmed_video_path, interim_output_directory, 'PP')
if IO.check_files(trimmed_video_path, preprocessed_files): # Checks if input files are missing, or output file(s) already exist, function returns True if this step can be run
    print('Starting Preprocessing...')
    isx.preprocess(trimmed_video_path, preprocessed_files)
    print('Preprocessing Finished!')


# STEP 7D: APPLY BANDPASS FILTER TO TRIMMED HD VIDEO FILES
bandpass_files = IO.make_isx_path(preprocessed_files, interim_output_directory, 'BP')
if IO.check_files(preprocessed_files, bandpass_files):
    print('Starting Bandpass Filtering...')
    isx.spatial_filter(preprocessed_files, bandpass_files, low_cutoff=spatial_bp_low_cutoff, high_cutoff=spatial_bp_high_cutoff)
    print('Bandpass Filtering Finished!')


# STEP 7E.1: GENERATE MEAN IMAGE TO USE AS THE REFERENCE FRAME FOR MOTION CORRECTION
mean_projection_file = IO.make_isx_path(trimmed_video_path, interim_output_directory, 'mean_image')
if IO.check_files(bandpass_files, mean_projection_file):
    print('Creating Mean Image...')
    isx.project_movie(bandpass_files, mean_projection_file[0], stat_type='mean')

# STEP 7E.2: APPLY MOTION CORRECTION TO TRIMMED HD VIDEO FILES
motion_correction_files = IO.make_isx_path(bandpass_files, interim_output_directory, 'MC')
translation_file = IO.make_isx_path(motion_correction_files, interim_output_directory, 'translations', 'csv')[0]
crop_rect_file = IO.make_isx_path([movie_stem], interim_output_directory, 'TRIMMED-crop_rect', 'csv')[0]

if IO.check_files(bandpass_files, [motion_correction_files, translation_file, crop_rect_file]):
    print('Starting Motion Correction...')
    isx.motion_correct(bandpass_files, motion_correction_files, max_translation=max_translation, reference_file_name=mean_projection_file[0],
                       low_bandpass_cutoff=motion_c_low_bandpass_cutoff, high_bandpass_cutoff=motion_c_high_bandpass_cutoff,
                       output_translation_files=translation_file, output_crop_rect_file=crop_rect_file)
    print('Motion Correction Finished!')


# STEP 7F: DELTA F/F NORMALIZATION
dff_files = IO.make_isx_path(motion_correction_files, interim_output_directory, 'DFF')
if IO.check_files(motion_correction_files, dff_files):
    print('Calculating DF/F...')
    isx.dff(motion_correction_files, dff_files, f0_type='mean')
    print('Calculating DF/F Finished!')


# STEP 7G: EXPORT TRIMMED, HD VIDEO
# Note: compression quality ranges from 0.001 (very low quality) to 1 (no compression); 0.4 seems to be a good medium
if IO.check_files(dff_files, movie_file_path):
    print('Exporting Movie to MP4...')
    isx.export_movie_to_mp4(dff_files, movie_file_path[0], compression_quality=0.4, write_invalid_frames=True)
    print('Exported HD Movie!')


# STEP 7H (OPTIONAL): DELETE ALL INTERIM FILES
if cleanup_interim_files:
    for each in preprocessed_files, bandpass_files, motion_correction_files, dff_files:
        Path(each[0]).unlink()
